# Miscellaneous functions for MAE

> miscellaneous functions for MAE

In [ ]:
#| default_exp mae.misc

In [2]:
#| export
import builtins
import datetime
import os
import time
from collections import defaultdict, deque
from pathlib import Path

import torch
import torch.distributed as dist
from torch import inf


In [57]:
#| export
from fastcore.foundation import *
from fastcore.basics import patch_to
from fastcore.test import *

In [11]:
#| export
class SmoothedValue(object):
    """Track a series of values and provide access to smoothed values over a
    window or the global series average.
    """

    def __init__(self, window_size=20, fmt=None):
        if fmt is None:
            fmt = "{median:.4f} ({global_avg:.4f})"
        self.deque = deque(maxlen=window_size)
        self.total = 0.0
        self.count = 0
        self.fmt = fmt

    def update(self, value, n=1):
        self.deque.append(value)
        self.count += n
        self.total += value * n

    def synchronize_between_processes(self):
        """
        Warning: does not synchronize the deque!
        """
        if not is_dist_avail_and_initialized():
            return
        t = torch.tensor([self.count, self.total], dtype=torch.float64, device='cuda')
        dist.barrier()
        dist.all_reduce(t)
        t = t.tolist()
        self.count = int(t[0])
        self.total = t[1]

    @property
    def median(self):
        d = torch.tensor(list(self.deque))
        return d.median().item()

    @property
    def avg(self):
        d = torch.tensor(list(self.deque), dtype=torch.float32)
        return d.mean().item()

    @property
    def global_avg(self):
        return self.total / self.count

    @property
    def max(self):
        return max(self.deque)

    @property
    def value(self):
        return self.deque[-1]

    def __str__(self):
        return self.fmt.format(
            median=self.median,
            avg=self.avg,
            global_avg=self.global_avg,
            max=self.max,
            value=self.value)

In [12]:
#| export
class MetricLogger(object):
    def __init__(self, delimiter="\t"):
        self.meters = defaultdict(SmoothedValue)
        self.delimiter = delimiter

In [17]:
 #| export
 @patch_to(MetricLogger)
 def update(self, **kwargs):
        for k, v in kwargs.items():
            if v is None:
                continue
            if isinstance(v, torch.Tensor):
                v = v.item()
            assert isinstance(v, (float, int))
            self.meters[k].update(v)


In [19]:
#| export
 # Test the update function
 
def test_update():
     logger = MetricLogger()
     
     # Test with float
     logger.update(loss=0.5)
     test_eq('loss' in logger.meters, True)
     test_eq(logger.meters['loss'].value, 0.5)
     
     # Test with int
     logger.update(epoch=1)
     test_eq('epoch' in logger.meters, True)
     test_eq(logger.meters['epoch'].value, 1)
     
     # Test with None (should be ignored)
     logger.update(ignored=None)
     test_eq('ignored' in logger.meters, False)
     
     # Test with tensor
     tensor_value = torch.tensor(1.5)
     logger.update(tensor_metric=tensor_value)
     test_eq('tensor_metric' in logger.meters, True)
     test_eq(logger.meters['tensor_metric'].value, 1.5)

 # Run the test
 test_update()

In [21]:
#| export
@patch_to(MetricLogger)
def __getattr__(self, attr):
    if attr in self.meters:
        return self.meters[attr]
    if attr in self.__dict__:
        return self.__dict__[attr]
    raise AttributeError("'{}' object has no attribute '{}'".format(
        type(self).__name__, attr))


In [22]:
#| export
def test_getattr():
    logger = MetricLogger()
    logger.update(loss=0.5, epoch=1)
    test_eq(logger.loss.value, 0.5)
    test_eq(logger.epoch.value, 1)
    test_eq(logger.meters['loss'].value, 0.5)
    test_eq(logger.meters['epoch'].value, 1)
    test_eq(logger.meters['loss'].avg, 0.5)
    test_eq(logger.meters['epoch'].avg, 1)

# Run the test
test_getattr()

In [25]:
#| export
@patch_to(MetricLogger)
def __str__(self):
    loss_str = []
    for name, meter in self.meters.items():
        loss_str.append(
            "{}: {}".format(name, str(meter))
        )
    return self.delimiter.join(loss_str)

In [27]:
#| export
def test_str():
	logger = MetricLogger()
	logger.update(loss=0.5, epoch=1)
	test_eq(str(logger), 'loss: 0.5000 (0.5000)\tepoch: 1.0000 (1.0000)')

In [28]:
#| export
@patch_to(MetricLogger)
def synchronize_between_processes(self):
        for meter in self.meters.values():
            meter.synchronize_between_processes()

In [30]:
#| export
def test_synchronize_between_processes():
    logger = MetricLogger()
    logger.update(loss=0.5, epoch=1)
    logger.synchronize_between_processes()
    test_eq(logger.loss.value, 0.5)
    test_eq(logger.epoch.value, 1)
    test_eq(logger.loss.global_avg, 0.5)
    test_eq(logger.epoch.global_avg, 1)

In [31]:
#| export
@patch_to(MetricLogger)
def add_meter(self, name, meter):
    self.meters[name] = meter
	


In [32]:
#| export
def test_add_meter():
    logger = MetricLogger()
    meter = SmoothedValue(window_size=10)
    logger.add_meter('new_meter', meter)
    test_eq(logger.meters['new_meter'], meter)
    

In [55]:
#| export
@patch_to(MetricLogger)
def log_every(
	self, 
	iterable, 
	print_freq, 
	header=None
	):
    i = 0
    if not header:
        header = ''
    start_time = time.time()
    end = time.time()
    iter_time = SmoothedValue(fmt='{avg:.4f}')
    data_time = SmoothedValue(fmt='{avg:.4f}')
    space_fmt = ':' + str(len(str(len(iterable)))) + 'd'
    log_msg = [
        header,
        '[{0' + space_fmt + '}/{1}]',
        'eta: {eta}',
        '{meters}',
        'time: {time}',
        'data: {data}'
    ]
    if torch.cuda.is_available():
        log_msg.append('max mem: {memory:.0f}')
    log_msg = self.delimiter.join(log_msg)
    MB = 1024.0 * 1024.0
    for obj in iterable:
        data_time.update(time.time() - end)
        yield obj
        iter_time.update(time.time() - end)
        if i % print_freq == 0 or i == len(iterable) - 1:
            eta_seconds = iter_time.global_avg * (len(iterable) - i)
            eta_string = str(datetime.timedelta(seconds=int(eta_seconds)))
            if torch.cuda.is_available():
                print(log_msg.format(
                    i, len(iterable), eta=eta_string,
                    meters=str(self),
                    time=str(iter_time), data=str(data_time),
                    memory=torch.cuda.max_memory_allocated() / MB))
            else:
                print(log_msg.format(
                    i, len(iterable), eta=eta_string,
                    meters=str(self),
                    time=str(iter_time), data=str(data_time)))
        i += 1
        end = time.time()
    total_time = time.time() - start_time
    total_time_str = str(datetime.timedelta(seconds=int(total_time)))
    if len(iterable) > 0:
        print('{} Total time: {} ({:.4f} s / it)'.format(
            header, total_time_str, total_time / len(iterable)))
    else:
        print('{} Total time: {} (0.0000 s / it)'.format(
		header, total_time_str))

In [62]:
#| export

def test_log_every():
    logger = MetricLogger()
    
    # Test with empty iterable
    empty_iterable = []
    # Avoid division by zero for empty iterable
    test_eq(list(logger.log_every(empty_iterable, 1)), [])
    
    # Test with single item iterable
    single_item = [1]
    test_eq(list(logger.log_every(single_item, 1)), [1])
    
    # Test with print_freq greater than iterable length
    small_iterable = [1, 2, 3]
    test_eq(list(logger.log_every(small_iterable, 10)), [1, 2, 3])
    
	# Test with None header
    list(logger.log_every(small_iterable, 1, header=None))
    
    # Test with custom header
    list(logger.log_every(small_iterable, 1, header="Custom Header"))
test_log_every() 

 Total time: 0:00:00 (0.0000 s / it)
	[0/1]	eta: 0:00:00		time: 0.0000	data: 0.0000
 Total time: 0:00:00 (0.0010 s / it)
	[0/3]	eta: 0:00:00		time: 0.0000	data: 0.0000
	[2/3]	eta: 0:00:00		time: 0.0000	data: 0.0000
 Total time: 0:00:00 (0.0001 s / it)
	[0/3]	eta: 0:00:00		time: 0.0000	data: 0.0000
	[1/3]	eta: 0:00:00		time: 0.0000	data: 0.0000
	[2/3]	eta: 0:00:00		time: 0.0000	data: 0.0000
 Total time: 0:00:00 (0.0001 s / it)
Custom Header	[0/3]	eta: 0:00:00		time: 0.0000	data: 0.0000
Custom Header	[1/3]	eta: 0:00:00		time: 0.0000	data: 0.0000
Custom Header	[2/3]	eta: 0:00:00		time: 0.0000	data: 0.0000
Custom Header Total time: 0:00:00 (0.0001 s / it)


# Distibuted Training Setup printing

 This function sets up a custom printing mechanism for distributed processing environments. Here's what it does:
 
 1. It takes a boolean parameter `is_master` to identify if the current process is the master process.
 2. It stores the built-in `print` function in `builtin_print`.
 3. It defines a new `print` function that:
    - Checks if printing should be forced (either explicitly or when there are more than 8 processes)
    - Only prints if it's the master process or if printing is forced
    - Adds a timestamp to each print statement
 4. It replaces the built-in `print` function with this new custom function.
 
 This setup ensures that in distributed environments, only the master process (or forced prints) will output to the console, reducing clutter and improving log readability.

In [ ]:
# | export

# Setup for distributed processing
def setup_for_distributed(is_master):
    "Disable printing for non-master processes in distributed setups"
    builtin_print = builtins.print

    def print(*args, **kwargs):
        force = kwargs.pop('force', False)
        force = force or (get_world_size() > 8)
        if is_master or force:
            now = datetime.datetime.now().time()
            builtin_print('[{}] '.format(now), end='')  # print with time stamp
            builtin_print(*args, **kwargs)

    builtins.print = print


In [63]:
#| export
def is_dist_avail_and_initialized():
    if not dist.is_available():
        return False
    if not dist.is_initialized():
        return False
    return True


 `is_dist_avail_and_initialized` is a nifty little function that does exactly what it says on the tin. It checks if distributed processing is both available and initialized. This is super handy when you're working with distributed systems and need to quickly verify if everything's set up and ready to go. It's a one-liner that combines two checks into a single, easy-to-read boolean. Remember, in Python, `and` short-circuits, so if `dist.is_available()` is False, it won't even bother checking `dist.is_initialized()`. Efficient and clear - just the way we like it!

In [65]:
#| export
def get_world_size():
    if not is_dist_avail_and_initialized():
        return 1
    return dist.get_world_size()

In [66]:
#| export
def get_rank():
    if not is_dist_avail_and_initialized():
        return 0
    return dist.get_rank()



In [67]:
#| export
# Test when distributed is not available or initialized
def test_get_rank_not_initialized():
    test_eq(get_rank(), 0)

# Note: We can't easily test the distributed case in this context,
# as it would require setting up a distributed environment.
# In a real-world scenario, you might want to add more tests
# for the distributed case in a separate test suite.

test_get_rank_not_initialized()

In [68]:
#| export
def is_main_process():
    return get_rank() == 0


def save_on_master(*args, **kwargs):
    if is_main_process():
        torch.save(*args, **kwargs)

In [69]:
#| export
def init_distributed_mode(args):
    if args.dist_on_itp:
        args.rank = int(os.environ['OMPI_COMM_WORLD_RANK'])
        args.world_size = int(os.environ['OMPI_COMM_WORLD_SIZE'])
        args.gpu = int(os.environ['OMPI_COMM_WORLD_LOCAL_RANK'])
        args.dist_url = "tcp://%s:%s" % (os.environ['MASTER_ADDR'], os.environ['MASTER_PORT'])
        os.environ['LOCAL_RANK'] = str(args.gpu)
        os.environ['RANK'] = str(args.rank)
        os.environ['WORLD_SIZE'] = str(args.world_size)
        # ["RANK", "WORLD_SIZE", "MASTER_ADDR", "MASTER_PORT", "LOCAL_RANK"]
    elif 'RANK' in os.environ and 'WORLD_SIZE' in os.environ:
        args.rank = int(os.environ["RANK"])
        args.world_size = int(os.environ['WORLD_SIZE'])
        args.gpu = int(os.environ['LOCAL_RANK'])
    elif 'SLURM_PROCID' in os.environ:
        args.rank = int(os.environ['SLURM_PROCID'])
        args.gpu = args.rank % torch.cuda.device_count()
    else:
        print('Not using distributed mode')
        setup_for_distributed(is_master=True)  # hack
        args.distributed = False
        return

    args.distributed = True

    torch.cuda.set_device(args.gpu)
    args.dist_backend = 'nccl'
    print('| distributed init (rank {}): {}, gpu {}'.format(
        args.rank, args.dist_url, args.gpu), flush=True)
    torch.distributed.init_process_group(backend=args.dist_backend, init_method=args.dist_url,
                                         world_size=args.world_size, rank=args.rank)
    torch.distributed.barrier()
    setup_for_distributed(args.rank == 0)

# Model Gradients and floating point precision

In [89]:
#| export
def get_grad_norm_(
	parameters, 
	norm_type: float = 2.0
	) -> torch.Tensor:
	"""
	Compute the norm of the gradients for a set of parameters.
	"""
	if isinstance(parameters, torch.Tensor):
		parameters = [parameters]

	parameters = [p for p in parameters if p.grad is not None]
	norm_type = float(norm_type)
	if len(parameters) == 0:
		return torch.tensor(0.)
	device = parameters[0].grad.device
	if norm_type == inf:
		total_norm = max(p.grad.detach().abs().max().to(device) for p in parameters)
	else:
		total_norm = torch.norm(torch.stack([torch.norm(p.grad.detach(), norm_type).to(device) for p in parameters]), norm_type)
	return total_norm


In [90]:
#| export

def test_get_grad_norm_():
	# Test case 1: Normal case with multiple parameters
	parameters = [torch.nn.Parameter(torch.randn(2, 2)) for _ in range(3)]
	for p in parameters:
		p.grad = torch.randn(2, 2)
	
	total_norm = get_grad_norm_(parameters, norm_type=2)
	expected_norm = torch.norm(torch.stack([torch.norm(p.grad.detach(), 2) for p in parameters]), 2)
	test_close(total_norm, expected_norm)

	# Test case 2: Single tensor parameter
	single_param = torch.nn.Parameter(torch.randn(2, 2))
	single_param.grad = torch.randn(2, 2)
	single_norm = get_grad_norm_(single_param, norm_type=2)
	expected_single_norm = torch.norm(single_param.grad.detach(), 2)
	test_close(single_norm, expected_single_norm)

	# Test case 3: Empty parameter list
	empty_norm = get_grad_norm_([])
	test_eq(empty_norm, torch.tensor(0.))

	# Test case 4: Infinity norm
	inf_norm = get_grad_norm_(parameters, norm_type=float('inf'))
	expected_inf_norm = max(p.grad.detach().abs().max() for p in parameters)
	test_close(inf_norm, expected_inf_norm)

	print("All tests passed!")

test_get_grad_norm_()

All tests passed!


In [92]:
#| export
def save_model(
    args, 
    epoch, 
    model, 
    model_without_ddp, 
    optimizer, 
    loss_scaler
    ):
    output_dir = Path(args.output_dir)
    epoch_name = str(epoch)
    if loss_scaler is not None:
        checkpoint_paths = [output_dir / ('checkpoint-%s.pth' % epoch_name)]
        for checkpoint_path in checkpoint_paths:
            to_save = {
                'model': model_without_ddp.state_dict(),
                'optimizer': optimizer.state_dict(),
                'epoch': epoch,
                'scaler': loss_scaler.state_dict(),
                'args': args,
            }

            save_on_master(to_save, checkpoint_path)
    else:
        client_state = {'epoch': epoch}
        model.save_checkpoint(
            save_dir=args.output_dir, 
            tag="checkpoint-%s" % epoch_name, 
            client_state=client_state
            )

# Test function

def test_save_model():
    # Mock objects and arguments
    class Args:
        output_dir = 'test_output'
    args = Args()
    epoch = 10
    model = MagicMock()
    model_without_ddp = MagicMock()
    # This line uses MagicMock from the unittest.mock module
    # To use it, you need to import it at the beginning of the file:
    # from unittest.mock import MagicMock
    model_without_ddp.state_dict.return_value = {'layer1': 'weights1'}
    optimizer = MagicMock()
    optimizer.state_dict.return_value = {'param_groups': []}
    loss_scaler = MagicMock()
    loss_scaler.state_dict.return_value = {'scale': 1.0}

    # Test case 1: With loss_scaler
    with patch('builtins.open', mock_open()) as mock_file, \
         patch('torch.save') as mock_save, \
         patch('os.makedirs') as mock_makedirs:
        save_model(args, epoch, model, model_without_ddp, optimizer, loss_scaler)
        mock_makedirs.assert_called_once_with('test_output', exist_ok=True)
        mock_save.assert_called_once()
        saved_dict = mock_save.call_args[0][0]
        test_eq(saved_dict['epoch'], 10)
        test_eq(saved_dict['model'], {'layer1': 'weights1'})
        test_eq(saved_dict['optimizer'], {'param_groups': []})
        test_eq(saved_dict['scaler'], {'scale': 1.0})
        test_eq(saved_dict['args'], args)

    # Test case 2: Without loss_scaler
    model.save_checkpoint = MagicMock()
    save_model(args, epoch, model, model_without_ddp, optimizer, None)
    model.save_checkpoint.assert_called_once_with(
        save_dir='test_output',
        tag="checkpoint-10",
        client_state={'epoch': 10}
    )

    print("All tests passed!")

test_save_model()

NameError: name 'MagicMock' is not defined